# Final Mask Creator

In [30]:
from jsonmerge import merge
import json
import os
import glob 
import shutil

import os
import glob

import cv2
import matplotlib.image as mpimg
import numpy as np
from math import degrees


def loadjsonfile(filepath):
    #Read JSON data into the datastore variable
    if filepath:
        with open(filepath, 'r', encoding='utf-8') as f:
            datastore = json.load(f)
            return datastore
        
def mergeJson(inputpath):
    
    result = None;
    for filename in glob.iglob(os.path.join(inputpath,'**/**.json'), recursive=True):
        
        jsonfile=loadjsonfile(filename)
        if(result==None):
            result=jsonfile
        else:
            result=merge(result, jsonfile)
        print("json items after mergeing %s : %d" %(filename, len(result.keys())))
    return result

def fix_labels(labels:dict):
    # creates dict of type ('label':idx) made from values from dict, NOT keys
    names = set([a.lower() for a in labels.values()])
    ids = list(range(len(names)))
    dictionary = dict(zip(names, ids))
    return dictionary

def create_labels(json:dict,wanted_labels = None ):
    '''
    creates labels for classes from dict using all options
    to counter duplicates in labels uses temporary dict from fix_labels for right mapping
    '''
    label_dict = json["_via_attributes"]['region']['Aorta']['options']
    labels = list(label_dict)
    real_idx = fix_labels(label_dict)
    #print(real_idx)
    result = dict()
    if wanted_labels is None:
        for label in labels:
            result[label] = real_idx[label_dict[label].lower()]
    else:
        for label in labels:
            full_label = label_dict[label].lower()
            if full_label in wanted_labels:
                result[label] = real_idx[label_dict[label].lower()]
        
    return result

def mask_creator(imageDict:dict,image_folder:str, label_dict:dict):
    '''
    Creates mask for for objects in iamgeDict that are in label_dict.
    Works for NOT crossing labels.
    '''
    
    imageName = imageDict['filename']
    try:
        image = mpimg.imread(os.path.join(image_folder,imageName))
    except OSError:
        return (None,None,False)
    
    size = image.shape[:2]
    
    mask = np.zeros(size)
    
    for region in imageDict['regions']:
        if region["region_attributes"].get("Aorta") in label_dict or region["region_attributes"].get("Struture") in label_dict:

            if region['shape_attributes']['name'] == 'polygon':
                
                x = region['shape_attributes']['all_points_x']
                y = region['shape_attributes']['all_points_y']
                poly = np.array(list(zip(x,y)))
                
                label = next(iter(region['region_attributes'].values()))
                
                mask = cv2.fillPoly(mask, [poly], label_dict[label])

            if region['shape_attributes']['name'] == 'ellipse':

                x = region['shape_attributes']['cx']
                y = region['shape_attributes']['cy']
                rx = region['shape_attributes']['rx']
                ry = region['shape_attributes']['ry']
                th = region['shape_attributes']['theta']
                
                label = next(iter(region['region_attributes'].values()))
                
                
                mask = cv2.ellipse(mask, (int(x),int(y)), (int(rx),int(ry)), degrees(th),0,360, label_dict[label], -1)
                
    # to check if something was drawn
    return (imageName,image,mask)
        
def prepare_single_patient(data_dir:str,target_dir:str,labels:dict = None):
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            with open(os.path.join(data_dir, file), "r", encoding='utf-8') as read_file:
                annotation = json.load(read_file)
    
    #wanted_labels = ['ascending aorta','descending aorta']
    
    
    patient_name = annotation['_via_settings']['project']['name']
    
    if labels is None:
        full_annotation = mergeJson(data_dir)
        labels = create_labels(full_annotation)
    
    im_dir = os.path.join(target_dir,'images')
    msk_dir = os.path.join(target_dir,'masks')
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    if not os.path.exists(im_dir):
        os.makedirs(im_dir)
    if not os.path.exists(msk_dir):
        os.makedirs(msk_dir)
    
    
    for i,(k,v) in enumerate(annotation['_via_img_metadata'].items()):
        print(patient_name,' ',k)
        image_mask = mask_creator(v,data_dir,labels)
        if np.any(image_mask[2]):
            im_name = os.path.join(im_dir,patient_name+'_'+str(i)+'.jpg')
            msk_name = os.path.join(msk_dir,patient_name+'_'+str(i)+'.jpg')
            cv2.imwrite(im_name, image_mask[1])
            cv2.imwrite(msk_name, image_mask[2])

def prepare_images(folders_dir:str,target_dir:str,wanted_labels:list):
    
    full_annotation = mergeJson(folders_dir)
    labels = create_labels(full_annotation,wanted_labels)
    
    for root, dirs, files in os.walk(folders_dir):
        for d in dirs:
            prepare_single_patient(os.path.join(root,d),target_dir,labels)

    with open(os.path.join(target_dir,'labels.json'), 'w') as fp:
        json.dump(labels, fp)
    
    return labels
    

In [ ]:
data = r"E:\University\ScienceWork\Medicine\AortaStuff\ML\datasets\data"
where = r"E:\University\ScienceWork\Medicine\AortaStuff\ML\datasets\desc_aorta"
wanted_labels = ['descending aorta']
prepare_images(data,where,wanted_labels)

# Mask=Channel version

In [31]:
from jsonmerge import merge
import json
import os
import glob 
import shutil

import os
import glob

import cv2
import matplotlib.image as mpimg
import numpy as np
from math import degrees


def loadjsonfile(filepath):
    #Read JSON data into the datastore variable
    if filepath:
        with open(filepath, 'r', encoding='utf-8') as f:
            datastore = json.load(f)
            return datastore
        
def mergeJson(inputpath):
    
    
    result = None
    for filename in glob.iglob(os.path.join(inputpath,'**/**.json'), recursive=True):
        
        jsonfile=loadjsonfile(filename)
        if(result==None):
            result=jsonfile
        else:
            result=merge(result, jsonfile)
        print("json items after mergeing %s : %d" %(filename, len(result.keys())))
    return result

def fix_labels(labels:dict):
    # creates dict of type ('label':idx) made from values from dict, NOT keys
    names = set([a.lower() for a in labels.values()])
    ids = list(range(len(names)))
    dictionary = dict(zip(names, ids))
    return dictionary

def create_labels(json:dict,wanted_labels = None ):
    '''
    creates labels for classes from dict using all options
    to counter duplicates in labels uses temporary dict from fix_labels for right mapping
    '''
    label_dict = json["_via_attributes"]['region']['Aorta']['options']
    labels = list(label_dict)
    real_idx = fix_labels(label_dict)
    #print(real_idx)
    result = dict()
    if wanted_labels is None:
        for label in labels:
            result[label] = real_idx[label_dict[label].lower()]
    else:
        for label in labels:
            full_label = label_dict[label].lower()
            if full_label in wanted_labels:
                result[label] = real_idx[label_dict[label].lower()]
        
    return result

def arr_sorter(arrays):
    max_vals = [np.max(a) for a in arrays]
    idx = np.argsort(max_vals)
    return np.take(arrays,idx,axis=0)

def mask_creator_npy(imageDict:dict,image_folder:str, label_dict:dict):
    '''
    Creates mask for for objects in imageDict that are in label_dict.
    Adds each mask into channel.
    '''
    
    imageName = imageDict['filename']
    # read image is possible, else return empty arrays
    try:
        image = mpimg.imread(os.path.join(image_folder,imageName))
    except OSError:
        return (None,None,False)
    
    size = image.shape[:2]
    
    mask = np.zeros(size)
    
    # saves also channel order
    mask_values = np.unique(list(label_dict.values()))
    masks = np.zeros([len(mask_values),size[0],size[1]])
    
    
    for region in imageDict['regions']:
        if region["region_attributes"].get("Aorta") in label_dict or region["region_attributes"].get("Struture") in label_dict:

            if region['shape_attributes']['name'] == 'polygon':
                
                x = region['shape_attributes']['all_points_x']
                y = region['shape_attributes']['all_points_y']
                poly = np.array(list(zip(x,y)))
                
                label = next(iter(region['region_attributes'].values()))
                
                mask = cv2.fillPoly(mask, [poly], label_dict[label])
                
                i = np.where(mask_values==label_dict[label])
                masks[i] = mask
                mask = np.zeros(size)

            if region['shape_attributes']['name'] == 'ellipse':

                x = region['shape_attributes']['cx']
                y = region['shape_attributes']['cy']
                rx = region['shape_attributes']['rx']
                ry = region['shape_attributes']['ry']
                th = region['shape_attributes']['theta']
                
                label = next(iter(region['region_attributes'].values()))
                
                
                mask = cv2.ellipse(mask, (int(x),int(y)), (int(rx),int(ry)), degrees(th),0,360, label_dict[label], -1)
                
                i = np.where(mask_values==label_dict[label])
                masks[i] = mask
                
                mask = np.zeros(size)
                
    # to check if something was drawn
    masks = np.transpose(masks,(1,2,0))
    return (imageName,image,masks)

def prepare_single_patient_npy(data_dir:str,target_dir:str,labels:dict = None):
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            with open(os.path.join(data_dir, file), "r", encoding='utf-8') as read_file:
                annotation = json.load(read_file)
    
    #wanted_labels = ['ascending aorta','descending aorta']
    
    
    patient_name = annotation['_via_settings']['project']['name']
    
    if labels is None:
        full_annotation = mergeJson(data_dir)
        labels = create_labels(full_annotation)
    
    im_dir = os.path.join(target_dir,'images')
    msk_dir = os.path.join(target_dir,'masks')
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    if not os.path.exists(im_dir):
        os.makedirs(im_dir)
    if not os.path.exists(msk_dir):
        os.makedirs(msk_dir)
    
    
    for i,(k,v) in enumerate(annotation['_via_img_metadata'].items()):
        print(patient_name,' ',k)
        image_mask = mask_creator_npy(v,data_dir,labels)
        if np.any(image_mask[2]):
            im_name = os.path.join(im_dir,patient_name+'_'+str(i)+'.jpg')
            msk_name = os.path.join(msk_dir,patient_name+'_'+str(i)+'.npy')
            cv2.imwrite(im_name, image_mask[1])
            with open(msk_name, 'wb') as f:
                np.save(f, image_mask[2])

def prepare_images_npy(folders_dir:str,target_dir:str,wanted_labels:list):
    
    full_annotation = mergeJson(folders_dir)
    labels = create_labels(full_annotation,wanted_labels) 

    for root, dirs, files in os.walk(folders_dir):
        for d in dirs:
            prepare_single_patient_npy(os.path.join(root,d),target_dir,labels)

    with open(os.path.join(target_dir,'labels.json'), 'w') as fp:
        json.dump(labels, fp)
    
    return labels
    

In [ ]:
data = r"E:\University\ScienceWork\Medicine\AortaStuff\ML\datasets\data"
where = r"E:\University\ScienceWork\Medicine\AortaStuff\ML\datasets\asc_aorta_npy"
wanted_labels = ['ascending aorta']
prepare_images_npy(data,where,wanted_labels)

# Mask=Channel version

In [8]:
from jsonmerge import merge
import json
import os
import glob 
import shutil

import os
import glob

import cv2
import matplotlib.image as mpimg
import numpy as np
from math import degrees


def loadjsonfile(filepath):
    #Read JSON data into the datastore variable
    if filepath:
        with open(filepath, 'r', encoding='utf-8') as f:
            datastore = json.load(f)
            return datastore
        
def mergeJson(inputpath):
    
    
    result = None
    for filename in glob.iglob(os.path.join(inputpath,'**/**.json'), recursive=True):
        
        jsonfile=loadjsonfile(filename)
        if(result==None):
            result=jsonfile
        else:
            result=merge(result, jsonfile)
        print("json items after mergeing %s : %d" %(filename, len(result.keys())))
    return result

def fix_labels(labels:dict):
    # creates dict of type ('label':idx) made from values from dict, NOT keys
    names = set([a.lower() for a in labels.values()])
    ids = list(range(len(names)))
    dictionary = dict(zip(names, ids))
    return dictionary

def create_labels(json:dict,wanted_labels = None ):
    '''
    creates labels for classes from dict using all options
    to counter duplicates in labels uses temporary dict from fix_labels for right mapping
    '''
    label_dict = json["_via_attributes"]['region']['Aorta']['options']
    labels = list(label_dict)
    real_idx = fix_labels(label_dict)
    #print(real_idx)
    result = dict()
    if wanted_labels is None:
        for label in labels:
            result[label] = real_idx[label_dict[label].lower()]
    else:
        for label in labels:
            full_label = label_dict[label].lower()
            if full_label in wanted_labels:
                result[label] = real_idx[label_dict[label].lower()]
        
    return result

def arr_sorter(arrays):
    max_vals = [np.max(a) for a in arrays]
    idx = np.argsort(max_vals)
    return np.take(arrays,idx,axis=0)

def mask_creator_npy(imageDict:dict,image_folder:str, label_dict:dict):
    '''
    Creates mask for for objects in imageDict that are in label_dict.
    Adds each mask into channel.
    '''
    
    imageName = imageDict['filename']
    # read image is possible, else return empty arrays
    try:
        image = mpimg.imread(os.path.join(image_folder,imageName))
    except OSError:
        return (None,None,False)
    
    size = image.shape[:2]
    
    mask = np.zeros(size)
    
    # saves also channel order
    mask_values = np.unique(list(label_dict.values()))
    masks = np.zeros([len(mask_values),size[0],size[1]])
    
    
    for region in imageDict['regions']:
        if region["region_attributes"].get("Aorta") in label_dict or region["region_attributes"].get("Struture") in label_dict:

            if region['shape_attributes']['name'] == 'polygon':
                
                x = region['shape_attributes']['all_points_x']
                y = region['shape_attributes']['all_points_y']
                poly = np.array(list(zip(x,y)))
                
                label = next(iter(region['region_attributes'].values()))
                
                mask = cv2.fillPoly(mask, [poly], label_dict[label])
                
                i = np.where(mask_values==label_dict[label])
                masks[i] = mask
                mask = np.zeros(size)

            if region['shape_attributes']['name'] == 'ellipse':

                x = region['shape_attributes']['cx']
                y = region['shape_attributes']['cy']
                rx = region['shape_attributes']['rx']
                ry = region['shape_attributes']['ry']
                th = region['shape_attributes']['theta']
                
                label = next(iter(region['region_attributes'].values()))
                
                
                mask = cv2.ellipse(mask, (int(x),int(y)), (int(rx),int(ry)), degrees(th),0,360, label_dict[label], -1)
                
                i = np.where(mask_values==label_dict[label])
                masks[i] = mask
                
                mask = np.zeros(size)
                
    # to check if something was drawn
    masks = np.transpose(masks,(1,2,0))
    return (imageName,image,masks)

def prepare_single_patient_npy(data_dir:str,target_dir:str,labels:dict = None):
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            with open(os.path.join(data_dir, file), "r", encoding='utf-8') as read_file:
                annotation = json.load(read_file)
    
    #wanted_labels = ['ascending aorta','descending aorta']
    
    
    patient_name = annotation['_via_settings']['project']['name']
    
    if labels is None:
        full_annotation = mergeJson(data_dir)
        labels = create_labels(full_annotation)
    
    im_dir = os.path.join(target_dir,'images')
    msk_dir = os.path.join(target_dir,'masks')
    
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    if not os.path.exists(im_dir):
        os.makedirs(im_dir)
    if not os.path.exists(msk_dir):
        os.makedirs(msk_dir)
    
    
    for i,(k,v) in enumerate(annotation['_via_img_metadata'].items()):
        print(patient_name,' ',k)
        image_mask = mask_creator_npy(v,data_dir,labels)
        if np.any(image_mask[2]):
            im_name = os.path.join(im_dir,patient_name+'_'+str(i)+'.jpg')
            msk_name = os.path.join(msk_dir,patient_name+'_'+str(i)+'.npy')
            cv2.imwrite(im_name, image_mask[1])
            with open(msk_name, 'wb') as f:
                np.save(f, image_mask[2])

def prepare_images_npy(folders_dir:str,target_dir:str,wanted_labels:list):
    
    full_annotation = mergeJson(folders_dir)
    labels = create_labels(full_annotation,wanted_labels) 

    for root, dirs, files in os.walk(folders_dir):
        for d in dirs:
            prepare_single_patient_npy(os.path.join(root,d),target_dir,labels)

    with open(os.path.join(target_dir,'labels.json'), 'w') as fp:
        json.dump(labels, fp)
    
    return labels
    